# Ontology interface

This tutorial demonstrates basic usages of __owlready2__ API for ontology manipulation. Notably, new ontology concept triple classes (subject, predicate, object) will be dynamically created, with optional existing ontology parent classes that are loaded from an OWL ontology. Then through the interconnected relations specified in triples, designators and their corresponding ontology concepts can be double-way queried for certain purposes, eg. making a robot motion plan. 

In [1]:
from pathlib import Path 
from typing import Optional, List, Type
import pycram
from pycram.designator import DesignatorDescription, ObjectDesignatorDescription

pybullet build time: Nov 28 2023 23:51:11
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
[WARN] [1712349614.148390]: Failed to import Giskard messages
[WARN] [1712349614.158395]: Could not import RoboKudo messages, RoboKudo interface could not be initialized


# Owlready2

[Owlready2](https://owlready2.readthedocs.io/en/latest/intro.html) is a Python package providing a transparent access to OWL ontologies. It supports various manipulation operations, including but not limited to loading, modification, saving ontologies. Built-in supported reasoners include [HermiT](http://www.hermit-reasoner.com) and [Pellet](https://github.com/stardog-union/pellet).

In [2]:
import logging
try:
    import owlready2
    from owlready2 import *
except ImportError:
    owlready2 = None
    logging.warn("Could not import owlready2, OWL Ontology Manager could not be initialized")

logging.getLogger().setLevel(logging.DEBUG)

# Ontology Manager

`OntologyManager` is the singleton class acting as the main interface between PyCram with ontologies, whereby object instances in the former could query relevant information based on the semantic connection with their corresponding ontology concepts.

Such connection, as represented by triples (subject-predicate-object), could be also created on the fly if not pre-existing in the loaded ontology.

Also new and updated concepts with their properties defined in runtime could be stored into an [SQLite3 file database](https://owlready2.readthedocs.io/en/latest/world.html) for reuse.

Here we will use [SOMA ontology](https://ease-crc.github.io/soma) as the baseline to utilize the generalized concepts provided by it.

In [3]:
from pycram.ontology import OntologyManager, SOMA_HOME_ONTOLOGY_IRI

ontology_manager = OntologyManager(SOMA_HOME_ONTOLOGY_IRI)
main_ontology = ontology_manager.main_ontology
soma = ontology_manager.soma
dul = ontology_manager.dul

[INFO] [1712349614.821621]: Main Ontology [http://www.ease-crc.org/ont/SOMA-HOME.owl#]'s name: SOMA-HOME has been loaded
[INFO] [1712349614.822448]: Main Ontology namespace: SOMA-HOME
[INFO] [1712349614.822881]: Loaded ontologies:
[INFO] [1712349614.823264]: http://www.ease-crc.org/ont/SOMA-HOME.owl#
[INFO] [1712349614.823646]: http://www.ease-crc.org/ont/DUL.owl#
[INFO] [1712349614.824033]: http://www.ease-crc.org/ont/SOMA.owl#


## Ontology Concept class
A built-in class named __`OntologyConcept`__, inheriting from __`owlready2.Thing`__, is created when `OntologyManager` is initialized, as the super class for all dynamically made ontology classes later on. It is then accessed by __`ontology_manager.main_ontology.OntologyConcept`__

Notable members:
- `designators`: a list of `DesignatorDescription` instances associated with the ontology concept
- `resolve`: a `Callable` returning a list of `DesignatorDescription`. It is used to provide which specific designators inferred from the ontology concept. Most typically, they are `designators`, but can be only a subset of it given certain conditions.  

## Query ontology classes and their properties

Classes in the loaded ontology can be queried based on their exact names, or part of them, or by namespace.
Here, we can see essential info (ancestors, super/sub-classes, properties, direct instances, etc.) of the found ontology class.

In [4]:
ontology_designed_container_class = ontology_manager.get_ontology_class('DesignedContainer')
ontology_manager.print_ontology_class(ontology_designed_container_class)
ontology_manager.get_ontology_classes_by_subname('PhysicalObject')
ontology_manager.get_ontology_classes_by_namespace('SOMA')

[INFO] [1712349614.944856]: -------------------
[INFO] [1712349614.945597]: SOMA.DesignedContainer <class 'owlready2.entity.ThingClass'>
[INFO] [1712349614.946100]: Super classes: [DUL.DesignedArtifact, DUL.DesignedArtifact, SOMA.hasDisposition.some(SOMA.Containment), SOMA.hasDisposition.some(SOMA.Containment)]
[INFO] [1712349614.946635]: Ancestors: {DUL.Object, DUL.Entity, owl.Thing, DUL.DesignedArtifact, SOMA.DesignedContainer, DUL.PhysicalArtifact, DUL.PhysicalObject}
[INFO] [1712349614.947097]: Subclasses: [SOMA.Bottle, SOMA.Crockery, SOMA.Box, SOMA.Building, SOMA.Carafe, SOMA.Cupboard, SOMA.Dishwasher, SOMA.Dispenser, SOMA.Drawer, SOMA.Jar, SOMA.Pack, SOMA.Oven, SOMA.Shaker, SOMA.Refrigerator, SOMA.TrashContainer]
[INFO] [1712349614.947604]: Properties: [rdf-schema.isDefinedBy, SOMA.hasDisposition, rdf-schema.comment, rdf-schema.label]
[INFO] [1712349615.005336]: Instances: []
[INFO] [1712349615.005853]: Direct Instances: []
[INFO] [1712349615.006306]: Inverse Restrictions: []
[IN

[SOMA.Affordance,
 SOMA.Disposition,
 SOMA.Setpoint,
 SOMA.Answer,
 SOMA.Message,
 SOMA.ProcessType,
 SOMA.OrderedElement,
 SOMA.System,
 SOMA.Binding,
 SOMA.Joint,
 SOMA.Color,
 SOMA.ExecutionStateRegion,
 SOMA.Feature,
 SOMA.FrictionAttribute,
 SOMA.SituationTransition,
 SOMA.NonmanifestedSituation,
 SOMA.JointLimit,
 SOMA.JointState,
 SOMA.Localization,
 SOMA.MassAttribute,
 SOMA.NetForce,
 SOMA.Succedence,
 SOMA.Preference,
 SOMA.Shape,
 SOMA.ShapeRegion,
 SOMA.SoftwareInstance,
 SOMA.StateType,
 SOMA.PhysicalEffector,
 SOMA.QueryingTask,
 SOMA.Order,
 SOMA.State,
 SOMA.Transient,
 SOMA.ColorRegion,
 SOMA.ForceAttribute,
 SOMA.API_Specification,
 SOMA.InterfaceSpecification,
 SOMA.AbductiveReasoning,
 SOMA.Reasoning,
 SOMA.Accessor,
 SOMA.Instrument,
 SOMA.Accident,
 SOMA.ActionExecutionPlan,
 SOMA.Actuating,
 SOMA.PhysicalTask,
 SOMA.AestheticDesign,
 SOMA.AffordsBeingSitOn,
 SOMA.CanBeSatOn,
 SOMA.SittingDestination,
 SOMA.CanSit,
 SOMA.AgentRole,
 SOMA.Sitting,
 SOMA.CausativeRo

__Descendants__ of an ontology class can be also queried by

In [5]:
ontology_manager.get_ontology_descendant_classes(ontology_designed_container_class)

[SOMA.Bottle,
 SOMA.DesignedContainer,
 SOMA.Bowl,
 SOMA.Crockery,
 SOMA.Box,
 SOMA.BreakfastPlate,
 SOMA.Plate,
 SOMA.Building,
 SOMA.Carafe,
 SOMA.CerealBox,
 SOMA.CoffeeCarafe,
 SOMA.Cup,
 SOMA.Cupboard,
 SOMA.DinnerPlate,
 SOMA.Dishwasher,
 SOMA.Dispenser,
 SOMA.Drawer,
 SOMA.Glass,
 SOMA.JamJar,
 SOMA.Jar,
 SOMA.KitchenCabinet,
 SOMA.MilkBottle,
 SOMA.MilkPack,
 SOMA.Pack,
 SOMA.NutellaJar,
 SOMA.Oven,
 SOMA.Pan,
 SOMA.PastaBowl,
 SOMA.PepperShaker,
 SOMA.Shaker,
 SOMA.Pot,
 SOMA.RaspberryJamJar,
 SOMA.Refrigerator,
 SOMA.SaladBowl,
 SOMA.SaltShaker,
 SOMA.SoupPot,
 SOMA.SugarDispenser,
 SOMA.TrashContainer,
 SOMA.Wardrobe,
 SOMA.WaterBottle,
 SOMA.WaterGlass,
 SOMA.WineBottle,
 SOMA.WineGlass]

## Create a new ontology class and its individual

A new ontology class can be created dynamically as inheriting from an existing class in the loaded ontology.
Here we create the class and its instance, also known as [__individual__](https://owlready2.readthedocs.io/en/latest/class.html#creating-equivalent-classes) in ontology terms.

In [6]:
ontology_custom_container_class = ontology_manager.create_ontology_concept_class('CustomContainerConcept',
                                                                            ontology_designed_container_class)
custom_container_concept = ontology_custom_container_class('ontology_custom_container_concept')

## Access ontology classes and individuals
All ontology classes created on the fly inherit from __`owlready2.Thing`__, and so share the same namespace with the loaded ontology instance `onto`. They can then be accessible through that namespace by __`main_ontology.<class_name>`__.
The same applies for individuals of those classes, accessible by __`main_ontology.<class_individual_name>`__

In [7]:
ontology_manager.print_ontology_class(main_ontology.OntologyConcept)
ontology_manager.print_ontology_class(main_ontology.CustomContainerConcept)
print(f"custom_container_concept is {main_ontology.ontology_custom_container_concept}: {custom_container_concept is main_ontology.ontology_custom_container_concept}")

[INFO] [1712349618.097436]: -------------------
[INFO] [1712349618.098242]: SOMA-HOME.OntologyConcept <class 'owlready2.entity.ThingClass'>
[INFO] [1712349618.098652]: Super classes: [owl.Thing]
[INFO] [1712349618.099044]: Ancestors: {owl.Thing, SOMA-HOME.OntologyConcept}
[INFO] [1712349618.099438]: Subclasses: [SOMA-HOME.CustomContainerConcept]
[INFO] [1712349618.099867]: Properties: []
[INFO] [1712349618.100529]: Instances: [SOMA-HOME.ontology_custom_container_concept]
[INFO] [1712349618.100943]: Direct Instances: []
[INFO] [1712349618.101336]: Inverse Restrictions: []
[INFO] [1712349618.101745]: -------------------
[INFO] [1712349618.102108]: SOMA-HOME.CustomContainerConcept <class 'owlready2.entity.ThingClass'>
[INFO] [1712349618.102462]: Super classes: [SOMA-HOME.OntologyConcept, SOMA.DesignedContainer]
[INFO] [1712349618.102863]: Ancestors: {SOMA-HOME.CustomContainerConcept, DUL.Object, DUL.PhysicalObject, DUL.Entity, DUL.DesignedArtifact, SOMA-HOME.OntologyConcept, SOMA.Designed

For ones already existing in the ontology, they can only be accessed through their corresponding ontology, eg: `soma` as follows

In [8]:
ontology_manager.print_ontology_class(soma.Cup)

[INFO] [1712349618.117187]: -------------------
[INFO] [1712349618.117756]: SOMA.Cup <class 'owlready2.entity.ThingClass'>
[INFO] [1712349618.118187]: Super classes: [SOMA.Crockery, SOMA.hasPhysicalComponent.some(SOMA.DesignedHandle)]
[INFO] [1712349618.118583]: Ancestors: {SOMA.Crockery, SOMA.Cup, SOMA.Tableware, DUL.Object, SOMA.DesignedTool, DUL.Entity, owl.Thing, DUL.DesignedArtifact, SOMA.DesignedContainer, DUL.PhysicalArtifact, DUL.PhysicalObject}
[INFO] [1712349618.118947]: Subclasses: []
[INFO] [1712349618.119378]: Properties: [SOMA.hasPhysicalComponent, rdf-schema.comment, rdf-schema.isDefinedBy]
[INFO] [1712349618.119986]: Instances: []
[INFO] [1712349618.120366]: Direct Instances: []
[INFO] [1712349618.120726]: Inverse Restrictions: []


## Connect ontology class individuals with designators
After creating `custom_container_concept` class, we connect it to a designator (say `obj_designator`) by:
- Append to `obj_designator.ontology_concepts` with `custom_container_concept`
- Append to `custom_container_concept.designators` with `obj_designator`

In [9]:
custom_container_designator = ObjectDesignatorDescription(names=["obj"])
custom_container_designator.ontology_concepts.append(custom_container_concept)
custom_container_concept.designators.append(custom_container_designator)

We can also automatize all the above setup with a single function call

In [10]:
another_custom_container_designator = ontology_manager.create_ontology_linked_designator(designator_name="another_custom_container",
                                                                                         designator_class=ObjectDesignatorDescription,
                                                                                         ontology_concept_name="AnotherCustomContainerConcept",
                                                                                         ontology_parent_class=ontology_designed_container_class)
print(another_custom_container_designator.ontology_concepts)
print(main_ontology.AnotherCustomContainerConcept.instances()[0].get_default_designator().names)

[SOMA-HOME.another_custom_container_concept]
['another_custom_container']


## Create new ontology triple classes

Concept classes of a triple, aka [__subject, predicate, object__], can be created dynamically. Here we will make an example creating ones for [__handheld objects__] and [__placeholder objects__], with a pair of predicate and inverse predicate signifying their mutual relation.

In [11]:
PLACEABLE_ON_PREDICATE_NAME = "placeable_on"
HOLD_OBJ_PREDICATE_NAME = "hold_obj"
ontology_manager.create_ontology_triple_classes(ontology_subject_parent_class=soma.DesignedContainer,
                                                subject_class_name="OntologyPlaceHolderObject",
                                                ontology_object_parent_class=soma.Shape,
                                                object_class_name="OntologyHandheldObject",
                                                predicate_name=PLACEABLE_ON_PREDICATE_NAME,
                                                inverse_predicate_name=HOLD_OBJ_PREDICATE_NAME,
                                                ontology_property_parent_class=soma.affordsBearer,
                                                ontology_inverse_property_parent_class=soma.isBearerAffordedBy)

There, we use `soma.DesignedContainer` & `soma.Shape`, existing concept in SOMA ontology, as the parent classes for the subject & object concepts respectively.
There is also a note that those classes, as inheriting from ##owlready2##-provided classes, are automatically given the namespace `onto`, so later on to be accessible through it.

Then now we define some instances of the newly created triple classes, and link them to object designators, again using __`ontology_manager.create_ontology_linked_designator()`__

In [12]:
def create_ontology_handheld_object(object_name: str, ontology_parent_class: Type[owlready2.Thing]):
    return ontology_manager.create_ontology_linked_designator(designator_name=object_name,
                                                              designator_class=ObjectDesignatorDescription,
                                                              ontology_concept_name=f"Onto{object_name}",
                                                              ontology_parent_class=ontology_parent_class)
# Holdable Objects
cookie_box = create_ontology_handheld_object("cookie_box", main_ontology.OntologyHandheldObject)
egg = create_ontology_handheld_object("egg", main_ontology.OntologyHandheldObject)
    
# Placeholder objects
placeholders = [create_ontology_handheld_object(object_name, main_ontology.OntologyPlaceHolderObject)
                for object_name in ['table', 'stool', 'shelf']]

egg_tray = create_ontology_handheld_object("egg_tray", main_ontology.OntologyPlaceHolderObject)

### Create ontology relations

Now we will create ontology relations or predicates between __placeholder objects__ and __handheld objects__ with __`ontology_manager.set_ontology_relation()`__

In [13]:
for place_holder in placeholders:
    ontology_manager.set_ontology_relation(subject_designator=cookie_box, object_designator=place_holder,
                                           predicate_name=PLACEABLE_ON_PREDICATE_NAME)

ontology_manager.set_ontology_relation(subject_designator=egg_tray, object_designator=egg,
                                       predicate_name=HOLD_OBJ_PREDICATE_NAME)

## Query designators based on their ontology-concept relations

Now we can make queries for designators from designators, based on the relation among their corresponding ontology concepts setup above

In [14]:
print(f"{cookie_box.names}'s placeholder candidates:",
      f"""{[placeholder.names for placeholder in
            ontology_manager.get_designators_by_subject_predicate(subject=cookie_box,
                                                                  predicate_name=PLACEABLE_ON_PREDICATE_NAME)]}""")

print(f"{egg.names}'s placeholder candidates:",
      f"""{[placeholder.names for placeholder in
            ontology_manager.get_designators_by_subject_predicate(subject=egg,
                                                                  predicate_name=PLACEABLE_ON_PREDICATE_NAME)]}""")

for place_holder in placeholders:
    print(f"{place_holder.names} can hold:",
          f"""{[placeholder.names for placeholder in
                ontology_manager.get_designators_by_subject_predicate(subject=place_holder,
                                                                      predicate_name=HOLD_OBJ_PREDICATE_NAME)]}""")

print(f"{egg_tray.names} can hold:",
      f"""{[placeholder.names for placeholder in
            ontology_manager.get_designators_by_subject_predicate(subject=egg_tray,
                                                                  predicate_name=HOLD_OBJ_PREDICATE_NAME)]}""")

['cookie_box']'s placeholder candidates: [['table'], ['stool'], ['shelf']]
['egg']'s placeholder candidates: [['egg_tray']]
['table'] can hold: [['cookie_box']]
['stool'] can hold: [['cookie_box']]
['shelf'] can hold: [['cookie_box']]
['egg_tray'] can hold: [['egg']]


# Practical examples

## Example 1
How about creating ontology concept classes encapsulating `pycram.enums.ObjectType`? We can do it by:

In [15]:
from pycram.enums import ObjectType

# Create a generic ontology concept class for edible objects
generic_edible_class = ontology_manager.create_ontology_concept_class('GenericEdible')

# Create a list of object designators sharing the same concept class as [generic_edible_class]
edible_obj_types = [ObjectType.MILK, ObjectType.BREAKFAST_CEREAL]
for object_type in ObjectType:
    if object_type in edible_obj_types:
        # Create a designator for the edible object
        ontology_manager.create_ontology_object_designator_from_type(object_type, generic_edible_class)

print(f'{generic_edible_class.name} object types:')
for edible_ontology_concept in generic_edible_class.direct_instances():
    print(edible_ontology_concept, [des.types for des in edible_ontology_concept.designators])


GenericEdible object types:
SOMA-HOME.milk_concept [['milk']]
SOMA-HOME.breakfast_cereal_concept [['breakfast_cereal']]


## Example 2
We could also make use of relations between ontology concepts that designators are associated with, to enable more abstract inputs in robot motion plan.

In a similar style to the scenario of __placeholder objects__ and __handheld objects__ above, but with a little bit difference, we will ask the robot to query which content holders (eg. cup, pitcher, bowl) whereby a milk box could be pourable into.

Basically, we will provide an ontology-based implementation for the query:
 
`abstract_ontology_concept -> specific_objects_in_world?`

To achieve it, we will create triple classes and configure a customized `resolve()` for the abstract concept, which returns its associated specific designators.
These designators are then used to again resolve for the target objects of interest, which become the inputs to a robot motion plan.

### Setup simulated environment

In [16]:
from pycram.bullet_world import BulletWorld, Object
from pycram.enums import ObjectType
from pycram.pose import Pose

from pycram.process_module import simulated_robot
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *

world = BulletWorld()
plane = BulletWorld.current_bullet_world.objects[0]
kitchen = Object("kitchen", ObjectType.ENVIRONMENT, "kitchen.urdf")
pr2 = Object("pr2", ObjectType.ROBOT, "pr2.urdf")
kitchen_designator = ObjectDesignatorDescription(names=["kitchen"])
robot_designator = ObjectDesignatorDescription(names=["pr2"]).resolve()

Scalar element defined multiple times: limit
Scalar element defined multiple times: limit


### Create PourableObject-LiquidHolder triple ontology classes

In [17]:
POURABLE_INTO_PREDICATE_NAME = "pourable_into"
HOLD_LIQUID_PREDICATE_NAME = "hold_liquid"
ontology_manager.create_ontology_triple_classes(ontology_subject_parent_class=soma.DesignedContainer,
                                                subject_class_name="OntologyLiquidHolderObject",
                                                ontology_object_parent_class=soma.Shape,
                                                object_class_name="OntologyPourableObject",
                                                predicate_name=POURABLE_INTO_PREDICATE_NAME,
                                                inverse_predicate_name=HOLD_LIQUID_PREDICATE_NAME,
                                                ontology_property_parent_class=soma.affordsBearer,
                                                ontology_inverse_property_parent_class=soma.isBearerAffordedBy)

### Spawn a pourable object & liquid holders into the world and Create their designators

In [18]:
# Holdable obj
milk_box = Object("milk_box", ObjectType.MILK, "milk.stl")
milk_box_designator = create_ontology_handheld_object(milk_box.name, main_ontology.OntologyPourableObject)

# Liquid-holders
cup = Object("cup", ObjectType.JEROEN_CUP, "jeroen_cup.stl", pose=Pose([1.4, 1, 0.9]))
bowl = Object("bowl", ObjectType.BOWL, "bowl.stl", pose=Pose([1.4, 0.5, 0.9]))
pitcher = Object("pitcher", ObjectType.GENERIC_OBJECT, "Static_MilkPitcher.stl", pose=Pose([1.4, 0, 0.9]))
milk_holders = [cup, bowl, pitcher]
milk_holder_designators = [create_ontology_handheld_object(obj.name, main_ontology.OntologyLiquidHolderObject)
                           for obj in milk_holders]

### Create an ontology relation between the designators of the pourable object & its liquid holders

In [19]:
for milk_holder_desig in milk_holder_designators:
    ontology_manager.set_ontology_relation(subject_designator=milk_box_designator, object_designator=milk_holder_desig,
                                           predicate_name=POURABLE_INTO_PREDICATE_NAME)

### Set up `resolve` for the ontology concept of the pourable object

In [20]:
milk_box_concept = milk_box_designator.get_default_ontology_concept()
def milk_box_concept_resolve(): 
    object_designator = ontology_manager.get_designators_by_subject_predicate(subject=milk_box_designator, predicate_name=POURABLE_INTO_PREDICATE_NAME)[0]
    return object_designator, object_designator.resolve()

milk_box_concept.resolve = milk_box_concept_resolve

Here, for demonstration purpose only, we specify the resolving result by __`milk_box_concept`__ as __`cup`__, the first-registered (default) pourable-into target milk holder, utilizing the ontology relation setup above.

Now, we can query the milk box's target liquid holder by resolving `milk_box_concept`

In [21]:
target_milk_holder_designator, target_milk_holder = milk_box_concept.resolve()
print('Pickup target object:', target_milk_holder.name)

Pickup target object: cup


### Robot picks up the target liquid holder

In [22]:
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([0.3]).resolve().perform()

    pickup_pose = CostmapLocation(target=target_milk_holder, reachable_for=robot_designator).resolve()
    pickup_arm = pickup_pose.reachable_arms[0]

    print(pickup_pose, pickup_arm)

    NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()

    PickUpAction(object_designator_description=target_milk_holder_designator, arms=[pickup_arm], grasps=["front"]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()

    place_island = SemanticCostmapLocation("kitchen_island_surface", kitchen_designator.resolve(), target_milk_holder_designator.resolve()).resolve()

    place_stand = CostmapLocation(place_island.pose, reachable_for=robot_designator, reachable_arm=pickup_arm).resolve()

    NavigateAction(target_locations=[place_stand.pose]).resolve().perform()

    PlaceAction(target_milk_holder_designator, target_locations=[place_island.pose], arms=[pickup_arm]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()
world.exit()

CostmapLocation.Location(pose=header: 
  seq: 0
  stamp: 
    secs: 1712349628
    nsecs: 307107686
  frame_id: "map"
pose: 
  position: 
    x: 0.6399999999999999
    y: 1.24
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.15234391170286138
    w: -0.988327543159185, reachable_arms=['left', 'right']) left


# Save ontologies to an OWL file
After all the above operations to our ontologies, we now can save them to an OWL file on disk

In [23]:
ontology_manager.save(f"{Path.home()}/ontologies/New{main_ontology.name}.owl")

[INFO] [1712349636.199589]: Ontologies have been saved to /home/ducthan/ontologies/NewSOMA-HOME.owl


# Optimize ontology loading with SQLite3
Upon the initial ontology loading from OWL, an SQLite3 file is automatically created, acting as the quadstore cache for the loaded ontologies. This allows them to be __selectively__ reusable the next time that being loaded.
More info can be referenced [here](https://owlready2.readthedocs.io/en/latest/world.html). 